In [1]:
from geoband.API import *
import pandas as pd
import folium
import json
import geopandas as gpd
from geojson import Feature, FeatureCollection, Point, dump
import matplotlib.pyplot as plt
import pydeck as pdk
import shapely
import jenkspy
import numpy as np
plt.rc("font", family = "Malgun Gothic")
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore') 

In [56]:
parking=GetCompasData('SBJ_2012_001', '1', '1.오산시_주정차단속(2018~2020).csv')
grid_accident=GetCompasData('SBJ_2012_001', '2', '2.오산시_어린이교통사고_격자.geojson')
grid_car_enrollment=GetCompasData('SBJ_2012_001', '3', '3.오산시_차량등록현황_격자.geojson')
pop=GetCompasData('SBJ_2012_001', '4', '4.오산시_연령별_거주인구격자(총인구).geojson')
junior_pop=GetCompasData('SBJ_2012_001', '5', '5.오산시_연령별_거주인구격자(유소년).geojson')
product_pop=GetCompasData('SBJ_2012_001', '6', '6.오산시_연령별_거주인구격자(생산가능인구).geojson')
senior_pop=GetCompasData('SBJ_2012_001', '7', '7.오산시_연령별_거주인구격자(고령).geojson')
floating_pop=GetCompasData('SBJ_2012_001', '8', '8.오산시_유동인구(2019).csv')
protection_area=GetCompasData('SBJ_2012_001', '9', '9.오산시_어린이보호구역.csv')
school=GetCompasData('SBJ_2012_001', '10', '10.오산시_학교위치정보.csv')
elementary_district=GetCompasData('SBJ_2012_001', '11', '11.오산시_초등학교_통학구.geojson')
middle_district=GetCompasData('SBJ_2012_001', '12', '12.오산시_중학교_학군.geojson')
kinder=GetCompasData('SBJ_2012_001', '13', '13.오산시_어린이집_유치원현황.csv')
weather=GetCompasData('SBJ_2012_001', '14', '14.오산시_기상데이터(2010~2019).csv')
traffic_cctv=GetCompasData('SBJ_2012_001', '15', '15.오산시_무인교통단속카메라.csv')
road_sign=GetCompasData('SBJ_2012_001', '16', '16.오산시_도로안전표지표준데이터.csv')
crosswalk=GetCompasData('SBJ_2012_001', '17', '17.오산시_횡단보도.geojson')
speed_bump=GetCompasData('SBJ_2012_001', '18', '18.오산시_과속방지턱표준데이터.csv')
traffic_sign=GetCompasData('SBJ_2012_001', '19', '19.오산시_신호등.geojson')
cctv=GetCompasData('SBJ_2012_001', '20', '20.오산시_CCTV설치현황.csv')
sidewalk=GetCompasData('SBJ_2012_001', '21', '21.오산시_인도.geojson')
bus_stop=GetCompasData('SBJ_2012_001', '22', '22.오산시_버스정류장.csv')
road=GetCompasData('SBJ_2012_001', '23', '23.오산시_상세도로망_LV6.geojson')
traffic=GetCompasData('SBJ_2012_001', '24', '24.평일_전일,시간대별_오산시_추정교통량_Level6.csv')
traffic_frequency=GetCompasData('SBJ_2012_001', '25', '25.평일_전일_오산시_혼잡빈도강도_Level6.csv')
traffic_time=GetCompasData('SBJ_2012_001', '26', '26.평일_전일_오산시_혼잡시간강도_Level6.csv')
building=GetCompasData('SBJ_2012_001', '27', '27.오산시_도로명주소_건물.geojson')
grid_building=GetCompasData('SBJ_2012_001', '28', '28.오산시_건물연면적_격자.geojson')
sports=GetCompasData('SBJ_2012_001', '29', '29.오산시_체육시설현황.csv')
academy=GetCompasData('SBJ_2012_001', '30', '30.오산시_학원_및_교습소_현황.csv')
le_sig=GetCompasData('SBJ_2012_001', '31', '31.오산시_법정경계(시군구).geojson')
ad_emd=GetCompasData('SBJ_2012_001', '32', '32.오산시_행정경계(읍면동).geojson')
le_emd=GetCompasData('SBJ_2012_001', '33', '33.오산시_법정경계(읍면동).geojson')
cadastral_map=GetCompasData('SBJ_2012_001', '34', '34.오산시_지적도.geojson')

In [2]:
parking=pd.read_csv('1.오산시_주정차단속(2018~2020).csv')
grid_accident=gpd.read_file('2.오산시_어린이교통사고_격자.geojson')
grid_car_enrollment=gpd.read_file('3.오산시_차량등록현황_격자.geojson')
pop=gpd.read_file('4.오산시_연령별_거주인구격자(총인구).geojson')
junior_pop=gpd.read_file('5.오산시_연령별_거주인구격자(유소년).geojson')
product_pop=gpd.read_file('6.오산시_연령별_거주인구격자(생산가능인구).geojson')
senior_pop=gpd.read_file('7.오산시_연령별_거주인구격자(고령).geojson')
floating_pop=pd.read_csv('8.오산시_유동인구(2019).csv')
protection_area=pd.read_csv('9.오산시_어린이보호구역.csv')
school=pd.read_csv('10.오산시_학교위치정보.csv')
elementary_district=gpd.read_file('11.오산시_초등학교_통학구.geojson')
middle_district=gpd.read_file('12.오산시_중학교_학군.geojson')
kinder=pd.read_csv('13.오산시_어린이집_유치원현황.csv')
weather=pd.read_csv('14.오산시_기상데이터(2010~2019).csv')
traffic_cctv=pd.read_csv('15.오산시_무인교통단속카메라.csv')
road_sign=pd.read_csv('16.오산시_도로안전표지표준데이터.csv')
crosswalk=gpd.read_file('17.오산시_횡단보도.geojson')
speed_bump=pd.read_csv('18.오산시_과속방지턱표준데이터.csv')
traffic_sign=gpd.read_file('19.오산시_신호등.geojson')
cctv=pd.read_csv('20.오산시_CCTV설치현황.csv')
sidewalk=gpd.read_file('21.오산시_인도.geojson')
bus_stop=pd.read_csv('22.오산시_버스정류장.csv')
road=gpd.read_file('23.오산시_상세도로망_LV6.geojson')
traffic=pd.read_csv('24.평일_전일,시간대별_오산시_추정교통량_Level6.csv')
traffic_frequency=pd.read_csv('25.평일_전일_오산시_혼잡빈도강도_Level6.csv')
traffic_time=pd.read_csv('26.평일_전일_오산시_혼잡시간강도_Level6.csv')
building=gpd.read_file('27.오산시_도로명주소_건물.geojson')
grid_building=gpd.read_file('28.오산시_건물연면적_격자.geojson')
sports=pd.read_csv('29.오산시_체육시설현황.csv')
academy=pd.read_csv('30.오산시_학원_및_교습소_현황.csv')
le_sig=gpd.read_file('31.오산시_법정경계(시군구).geojson')
ad_emd=gpd.read_file('32.오산시_행정경계(읍면동).geojson')
le_emd=gpd.read_file('33.오산시_법정경계(읍면동).geojson')

In [3]:
#지도 그리는 함수 지정
def drawing_Choro(df, osan_grid, title, legend):
    center=[37.1498, 127.0772]
    # 맵이 center 에 위치하고, zoom 레벨은 16로 시작하는 맵 m을 만듭니다.
    m1 = folium.Map(
        location=center,
        zoom_start=13,
        tiles='http://api.vworld.kr/req/wmts/1.0.0/26BFD173-330E-3D54-9C39-895DDD8F23B3/Base/{z}/{y}/{x}.png',
        attr='My Data Attribution'
    )

    # Choropleth 레이어를 만들고, 맵 m에 추가합니다.
    folium.Choropleth(
        geo_data=osan_grid,
        data=df,
        columns=('gid', 'point'),
        key_on='feature.properties.gid',
        legend_name = legend,
        fill_color='PuBuGn',
        line_opacity = 0.2,
    ).add_to(m1)

    # 맵 m을 출력합니다.
    m1.save("{0}.html".format(title))

In [4]:
#지도에 맵핑할 격자들 만들기
osan_grid=pop.drop('val',axis=1)
osan_grid

# pydeck 쓰기 위한 데이터 처리 함수 지정 

In [5]:
#주정차 단속
parking.columns=['date','parking_location','ad_area','school_zone',
                 'parking_lon','parking_lat','parking_type']
#스쿨존에 해당하면 1 아니면 0
parking.loc[parking['school_zone']=='N','school_zone']=0
parking.loc[parking['school_zone']=='Y','school_zone']=1
parking

In [ ]:
#parking null값에 해당하는 곳은 유탑유블레스 관련한 cctv들임 어떻게 처리할까-SY
parking[parking['parking_lon'].isnull()]

##### 창균 (12.22) #####
# 오산시 유탑유블레스 남측 좌표 
parking["parking_lon"].fillna(127.072291, inplace = True)
parking["parking_lat"].fillna(37.141363, inplace = True)
parking

In [ ]:
#나중에 비율 구할때 써야됨
#val값에 null은 측정이 안된 곳이기에 0으로 처리해줌-SY
pop=pop.rename(columns={'val':'pop_val'})
pop=pop.fillna(0)
junior_pop=junior_pop.rename(columns={'val':'junior_val'})
junior_pop=junior_pop.fillna(0)
senior_pop=senior_pop.rename(columns={'val':'senior_val'})
senior_pop=senior_pop.fillna(0)
product_pop=product_pop.rename(columns={'val':'product_val'})
product_pop=product_pop.fillna(0)

In [ ]:
# 정민 수정사항(12/21)
# 오산시 격자별 유소년 인구수/총 인구수 데이터 전처리

junior_ratio_pop = junior_pop.merge(pop, on ='gid')
junior_ratio_pop = junior_ratio_pop.drop(['geometry_x'], axis = 1)
junior_ratio_pop.columns = ['gid', 'junior_val', 'pop_val', 'geometry']
junior_ratio_pop['junior_ratio'] = junior_ratio_pop['junior_val']/junior_ratio_pop['pop_val'] * 100

junior_ratio_pop['junior_ratio'].value_counts()
junior_ratio_pop=junior_ratio_pop.fillna(0)
junior_ratio_pop

In [ ]:
# 주니어 인구비율 분포확인
%matplotlib inline
junior_ratio_pop["junior_ratio"].plot(xlabel = "gid", ylabel = "junior population ratio to total population")

In [ ]:
#합치는 코드 있어서 한번만 돌려야됨
#유동인구
col_list=list(floating_pop)
col_list_all=col_list[3:]
col_list_14_20=col_list[17:23]
#모든 시간대 유동인구
floating_pop['all']=floating_pop[col_list_all].sum(axis=1)
#사고가 가장 많이 일어나는 14~20
floating_pop['14_20']=floating_pop[col_list_14_20].sum(axis=1)
floating_pop=floating_pop.rename(columns={'STD_YM':'YM','lon':'floating_lon','lat':'floating_lat'})
floating_pop

In [ ]:
#어린이 보호구역
protection_area.columns=['protection_type','protection_name','protection_cctv',
                         'protection_cctv_count','width','protection_lon','protection_lat']
#cctv 있으면 1 없으면 0
protection_area.loc[protection_area['protection_cctv']=='N','protection_cctv']=0
protection_area.loc[protection_area['protection_cctv']=='Y','protection_cctv']=1
protection_area

In [ ]:
#학교 위치정보
a=school.loc[school['학교구분'].isin(['고등학교','중학교'])].index
school=school.drop(a)
school.columns=['school_type','school_name','school_lon','school_lat']
school

In [ ]:
# 초등학교 통학구
#type이 0이면 단독 통학구 1이면 다중 통학구?
elementary_district.columns=['objectid','district_id','ed_name','ed_type','ed_geometry']
elementary_district.head()


In [ ]:
# 어린이집 유치원현황
kinder=kinder.drop('시설타입',axis=1)
kinder.columns=['kinder_type','kinder_name','kinder_lon','kinder_lat']
kinder

In [ ]:
#무인교통 단속 카메라
traffic_cctv=traffic_cctv.drop(['도로노선방향','단속구분'],axis=1)
traffic_cctv.columns=['tc_road_name','tc_location_name','tc_lon','tc_lat']
traffic_cctv

In [ ]:
#도로안전표지 표준데이터
road_sign.columns=['road_sign_num','road_sign_type','road_sign_lon','road_sign_lat']
road_sign

In [ ]:
#횡단보도
## 창균 추가 수정 (12.22)
crosswalk.columns=['crosswalk_objectid','crosswalk_geometry']
crosswalk

In [ ]:
#과속방지턷표준데이터
speed_bump.columns=['bump_road_name','bump_location_name','bump_height','bump_width',
                   'bump_length','bump_sep','bump_continue','bump_lon','bump_lat']
#과속방지턱 보차분리여부,연속형 여부 0,1로 변환
speed_bump.loc[speed_bump['bump_sep']=='N','bump_sep']=0
speed_bump.loc[speed_bump['bump_sep']=='Y','bump_sep']=1
speed_bump.loc[speed_bump['bump_continue']=='N','bump_continue']=0
speed_bump.loc[speed_bump['bump_continue']=='Y','bump_continue']=1
speed_bump


In [ ]:
#신호등
## 창균 추가 수정 (12.22)
traffic_sign.columns=['traffic_sign_objectid','traffic_sign_geometry']
traffic_sign

In [ ]:
#cctv
# cctv_type A: 방범 B:도시공원 C:어린이보호 D:차량방범
cctv.columns=['cctv_type','cctv_road_name','cctv_lat','cctv_lon']
#cctv_lat,lon에서 중복적으로 결측값이 나와서 drop, road_name도 null값 nan으로 처리-SY
cctv=cctv.drop([500,567],axis=0)
cctv=cctv.fillna('NaN')
cctv

In [ ]:
#인도
sidewalk=sidewalk.drop('QUAL',axis=1)
sidewalk.columns=['sidewalk_id','sidewalk_bicycle',
                  'sidewalk_type','sidewalk_geometry']
#자전거 도로유무 BYC001이 있고 나머지가 없음
sidewalk.loc[sidewalk['sidewalk_bicycle']=='BYC001','sidewalk_bicycle']=1
sidewalk.loc[sidewalk['sidewalk_bicycle']=='BYC002','sidewalk_bicycle']=0
sidewalk

In [ ]:
# 버스 정류장
## 창균 추가 수정 (12.22)
bus_stop.columns=['bus_stop_id','bus_stop_name','bus_stop_lon','bus_stop_lat']
bus_stop

In [ ]:
#상세도로망
#road_name null값 nan으로 처리-SY
road=road[road['road_rank'].isin(['103','105','106','107'])]
road=road.drop('facil_name',axis=1)
road=road.fillna('NaN')
road

In [ ]:
traffic

In [ ]:
#평일 시간대별 추정교통량
#road_name null값 nan으로 처리-SY
traffic=traffic[traffic['도로등급'].isin(['103','105','106','107'])]
traffic=traffic.drop(['평일주말','시도명','시군구명'],axis=1)
traffic=traffic.fillna('NaN')
traffic.columns=['link_id','road_rank','link_length','road_name',
               'emd_name','time','all_traffic','car_traffic',
                'bus_traffic','freight_car_traffic']
traffic

In [ ]:
# 평일 혼잡빈도강도
#road_name null값 nan으로 처리-SY
traffic_frequency=traffic_frequency[traffic_frequency['도로등급'].isin(['103','105','106','107'])]
traffic_frequency=traffic_frequency.drop(['시도명','시군구명','평일주말','시간적범위'],axis=1)
traffic_frequency=traffic_frequency.fillna('nan')
traffic_frequency.columns=['link_id','road_rank','link_length','road_name',
                          'emd_name','frequency']
traffic_frequency

In [ ]:
#평일 혼잡시간강도
#road_name null값 nan으로 처리-SY
traffic_time=traffic_time[traffic_time['도로등급'].isin(['103','105','106','107'])]
traffic_time=traffic_time.drop(['시도명','시군구명','평일주말','시간적범위'],axis=1)
traffic_time=traffic_time.fillna('NaN')
traffic_time.columns=['link_id','road_rank','link_length','road_name',
                          'emd_name','time']
traffic_time

In [ ]:
#건물
building=building.drop(['UND_FLO_CO','GRO_FLO_CO','BULD_NM_DC','BULD_NM'],axis=1)
building=building.rename(columns={'geometry':'building_geometry'})
building

In [ ]:
#격자별 건물연견적
#val값의 null은 측정이 안된 값이라 0으로 처리-SY
grid_building.columns=['gid','grid_building_val','grid_building_geometry']
grid_building=grid_building.fillna(0)
grid_building

In [ ]:
#체육시설
sports.columns=['sports_type','sports_lon','sports_lat']
sports

In [ ]:
#학원,교습소
academy=academy.drop('교습과정명',axis=1)
academy.columns=['academy_type','academy_lon','academy_lat']
academy=academy.fillna('NaN')
academy

# 격자에 대한 전처리 

In [ ]:
#각 격자에 교통사고 횟수 concat
osan_grid_info=pd.concat([osan_grid,grid_accident['accident_cnt']],axis=1)
#각 격자에 거주인구수 concat
osan_grid_info=pd.concat([osan_grid_info,pop['pop_val']],axis=1)
#각 격자에 유소년 거주인구수 concat
osan_grid_info=pd.concat([osan_grid_info,junior_pop['junior_val']],axis=1)
#각 격자에 건물 면적 concat
osan_grid_info=pd.concat([osan_grid_info,grid_building['grid_building_val']],axis=1)
osan_grid_info